In [1]:
import xml.etree.ElementTree as ET
import os
import numpy as np
import subprocess
from tabulate import tabulate
import matplotlib.pyplot as plt
import pandas as pd
from random import randrange
from collections import defaultdict
from enum import Enum
from numpy.random import choice
from random import sample

In [2]:
bm_folder = '../benchmarks/'
file_name = 'sparcT1_core_stratixiv_arch_timing'
path_blif = os.path.join(bm_folder, 'blif',file_name +'.blif')
path_graphfiles_folder = os.path.join(bm_folder,'graphfile' ,file_name)
arch_name = 'stratixiv_arch.timing.xml'
path_arch_folder = os.path.join(bm_folder,'architecture' ,arch_name)
path_log_file = os.path.join(bm_folder, 'misc','vpr_stdout' +'.log')
hmetis = 'hmetis-1.5-linux/hmetis'
if not os.path.isdir(path_graphfiles_folder):
    os.mkdir(path_graphfiles_folder)

tree = ET.parse(path_arch_folder)
root = tree.getroot()

### Read original blif

In [3]:
file1 = open(path_blif, 'r')
lines = file1.readlines()
file1.close()

#### Parse subckt models

In [4]:
models = []
index1 = 0
for index2, line in enumerate(lines):
    if '.model' in line:
        models.append(lines[index1:index2])
        index1 = index2
models.append(lines[index1:])
main_model = models[1]
subckt_models = models[2:]

In [5]:
class Block:
    def __init__(self, type_, sub_type, ins, clks, outs):
        self.sub_type = sub_type
        self.type = type_
        self.inputs = ins  #port:net
        self.outputs = outs
        self.clocks = clks
        
    def add_input(self, port, net):
        self.inputs[port] = net
        
    def add_output(self, port, net):
        self.outputs[port] = net
    
    def add_clk_port(self, port):
        self.clocks.append(port)
        
        
class Netlist:
    def __init__(self):
        self.blocks = []
        self.nets = None
        
    def add_block(self, block):
        self.blocks.append(block)
        
    def net_graph(self):
        self.nets = defaultdict(lambda: defaultdict(list))
        
        for block_num, block in enumerate(self.blocks):
            subckt_type = block.type
#             if subckt_type != 'dffeas':
            for port, input_ in block.inputs.items():
                self.nets[input_]['sinks'].append({block_num: port})
            for port, output_ in block.outputs.items():
                self.nets[output_]['source'].append({block_num: port})
                    
    def get_net_graph(self):
        if not self.nets:
            self.net_graph()
        return self.nets
    
    def get_block_graph(self):
        return self.blocks
    
    def get_blocks_with_sub_type(self, sub_type):
        return [block for block in self.blocks if block.sub_type == sub_type]
    
    
    
class Subckt:
    def __init__(self, type_, inputs, outputs):
        self.data_inputs = []
        self.ctrl_inputs = []
        self.inputs = inputs
        self.outputs = outputs
        self.type = type_

 
class PortLabel(Enum):
    GNL = 1
    CTRL = 2
    CARRYCHAIN = 3
    CONSTANT = 4
    
class SubcktLabel(Enum):
    RAM = 1
    LAB = 2
    MAC = 3

class Subckt_instance:
    def __init__(self, type_, in_ports, clocks, out_ports):
        self.type = type_   
        self.in_ports = dict(zip(in_ports, len(in_ports) * [PortLabel.GNL]))
        self.out_ports = dict(zip(out_ports, len(out_ports) * [PortLabel.GNL]))
        self.clocks = clocks
        if 'ram' in self.type:
            self.label = SubcktLabel.RAM
        elif 'mac' in self.type:
            self.label = SubcktLabel.MAC
        else:
            self.label = SubcktLabel.LAB
    
    
class Subckt_instances:
    def __init__(self):
        self.subckts = {}
        self.count = defaultdict(int)
        
    def add_subckt(self, subckt):
        name = self.identify_subckt(subckt)
        if name == None:
            name = f'm{len(self.subckts):03}'
            self.subckts[name] = subckt
        self.count[name] = self.count[name] + 1
        return name
        
    def identify_subckt(self, subckt):
        for name, subckt_ in self.subckts.items():
            if subckt_.type == subckt.type:
                if subckt_.in_ports == subckt.in_ports:
                    if subckt_.out_ports == subckt.out_ports:
                        if subckt_.clocks == subckt.clocks:
                            return name
        return None
    
    def get_instances_with_label(self, label):
        return {sub_type: subckt for sub_type, subckt in self.subckts.items() if subckt.label == label}

In [6]:
arch_subckts = {}

for subckt_model in subckt_models:
    input_line, output_line = False, False
    inputs = []
    outputs = []
    for line in subckt_model:
        if '.model' in line:
            name = line.split()[1]
        if '.inputs' in line:
            input_line = True
            output_line = False
        elif '.outputs' in line:
            input_line = False
            output_line = True 
        elif line[0] == '.':
            input_line = False
            output_line = False
        elif input_line:
            inputs.append(line.split()[0])
        elif output_line:
            outputs.append(line.split()[0])
    
    arch_subckts[name] = Subckt(name, inputs, outputs)
    
for subckt in arch_subckts.values():
    for input_ in subckt.inputs:
        if 'data' in input_:
            subckt.data_inputs.append(input_)
        else:
            subckt.ctrl_inputs.append(input_)
    
            
arch_subckts['dffeas'].ctrl_inputs.remove('d')
arch_subckts['dffeas'].ctrl_inputs.remove('ena')
arch_subckts['dffeas'].data_inputs.append('d')
arch_subckts['dffeas'].data_inputs.append('ena')

### Collect data

In [7]:
new_main_model = []
new_line_split = []

# Format
for line in main_model:
    line_split = line.split()
    if line_split != [] and (line_split[-1] == '\\'):
        new_line_split.extend(line_split[:-1])
    else:
        new_line_split.extend(line_split)
        new_line = ' '.join(new_line_split)
        new_main_model.append(new_line_split)
        new_line_split = []
        
# collect .inputs, .outputs, .names, .latch, .subckt
inputs_main = None
outputs_main = None
names_main = []
latches_main = []
subckts_main = []

for split_line in new_main_model:
    if split_line == []:
        pass
    elif split_line[0] == '.inputs':
        inputs_main = split_line
    elif split_line[0] == '.outputs':
        outputs_main = split_line
    elif split_line[0] == '.names': 
        names_main.append(split_line)
    elif split_line[0] == '.latch':
        latches_main.append(split_line)
    elif split_line[0] == '.subckt':
        subckts_main.append(split_line)

In [8]:
netlist = Netlist()
subckt_instances = Subckt_instances()

for subckt in subckts_main:
        #subckt first: '.subckt', second: 'type_', then 'IO_name=netname'
        type_ = subckt[1]
        ins = {}
        clks = []
        outs = {}
        for io_net in subckt[2:]:
            port, net = io_net.split('=')
            if port in arch_subckts[type_].inputs:
                if 'clk' in port:
                    clks.append(port)
                else:
                    ins[port] = net
            elif port in arch_subckts[type_].outputs:
                outs[port]= net
        
        input_ports = list(ins)
        output_ports = list(outs)
        
        subckt_instance = Subckt_instance(type_, input_ports, clks, output_ports)
        instance_type = subckt_instances.add_subckt(subckt_instance)
        
        netlist.add_block(Block(type_, instance_type, ins, clks, outs))

### Trim subsckt instances
1. Carrychains
2. Vcc nets, gnd nets
3. ctrl nets

1) Ctrl nets

Collect ctrl ports

In [9]:
ctrl_nets = set()
ctrl_ports = set()
ctrl_ports_nets = defaultdict(lambda: defaultdict(int))
for block in netlist.blocks:
    ctrl_inputs = arch_subckts[block.type].ctrl_inputs
    for port, net in block.inputs.items():
        if port in ctrl_inputs and port not in ['cin', 'cout', 'sharein', 'shareout']:
            ctrl_ports.add(port)
            ctrl_nets.add(net)
            ctrl_ports_nets[port][net] += 1
ctrl_nets.remove('gnd')
ctrl_nets.remove('vcc')

Mark ctrl ports in subckt instances

In [10]:
for subckt in subckt_instances.subckts.values():
    for in_port in subckt.in_ports.keys():
        if in_port in ctrl_ports:
            subckt.in_ports[in_port] = PortLabel.CTRL

Ctrl nets for RAM blocks are grouped, call them ctrl_signatures_ram and collect

In [11]:
ram_ctrl_signatures = defaultdict(list)
ram_ctrl_signatures_counts = defaultdict(list)

for sub_type, subckt in subckt_instances.get_instances_with_label(SubcktLabel.RAM).items():
    sign_index = {}    
    for block in netlist.get_blocks_with_sub_type(sub_type):

        sign = ''
        ram_ctrl_signature = {}
        for in_port, port_label in subckt.in_ports.items():
            if port_label == PortLabel.CTRL:
                ram_ctrl_signature[in_port] = block.inputs[in_port]
                sign += block.inputs[in_port]
                
        if ram_ctrl_signature not in ram_ctrl_signatures[sub_type]:
            ram_ctrl_signatures[sub_type].append(ram_ctrl_signature)
            sign_index[sign] = len(sign_index)
            ram_ctrl_signatures_counts[sub_type].append(1)
        else:
            ram_ctrl_signatures_counts[sub_type][sign_index[sign]] += 1

2) Constant nets

gnd: signb, signa \
vcc: portare, ena2, ena3

In [12]:
constant_ports = {
    'signb': 'gnd',
    'signa': 'gnd',
    'portare': 'vcc',
    'ena2': 'vcc',
    'ena3': 'vcc'
}

Mark constant ports in subckt instances

In [13]:
for subckt in subckt_instances.subckts.values():
    for in_port in subckt.in_ports.keys():
        if in_port in constant_ports:
            subckt.in_ports[in_port] = PortLabel.CONSTANT

3) Carry chains

Identify carry chain circuits and mark port

In [14]:
simple_chain_subckt_names = []
double_chain_subckt_names = []

for name, subckt in subckt_instances.subckts.items():
    if 'cin' in subckt.in_ports:
        subckt.in_ports['cin'] = PortLabel.CARRYCHAIN
        if 'sharein' in subckt.in_ports:
            double_chain_subckt_names.append(name)
            subckt.in_ports['sharein'] = PortLabel.CARRYCHAIN
        else:
            simple_chain_subckt_names.append(name)
        if 'cout' in subckt.out_ports:
            subckt.out_ports['cout'] = PortLabel.CARRYCHAIN
        if 'shareout' in subckt.out_ports:
            subckt.out_ports['shareout'] = PortLabel.CARRYCHAIN

Identify forbidden blocks: two LUTs in one LAB can have max 7 different inputs -> for simplicity don't generate blocks with more than 3 datainputs

In [15]:
forbidden_subckt_names = []
for subckt_name in simple_chain_subckt_names:
    subckt = subckt_instances.subckts[subckt_name]
    count = subckt_instances.count[subckt_name]
    if len(subckt.in_ports) == 4:
        subckt_3_ports = subckt_name
    if len(subckt.in_ports) > 4:
        forbidden_subckt_names.append(subckt_name)
        print(f'Block {subckt_name} with {len(subckt.in_ports) - 1} dataports occurs {count} time(s)')
        
for forbidden_subckt_name in forbidden_subckt_names:
    count = subckt_instances.count[forbidden_subckt_name]
    subckt_instances.count[subckt_3_ports] += count
    subckt_instances.count[forbidden_subckt_name] = 0
    
# Double chain subckts have max 4 ports in this example.. 
for subckt_name in double_chain_subckt_names:
    subckt = subckt_instances.subckts[subckt_name]
    count = subckt_instances.count[subckt_name]
    assert len(subckt.in_ports) <= 5

Block m052 with 5 dataports occurs 67 time(s)
Block m062 with 4 dataports occurs 1 time(s)


Collect carry chains length distribution

In [16]:
# net --> atoms
carry_chain_nets = defaultdict(list)
for block_num, block in enumerate(netlist.get_block_graph()):
    if block.sub_type in simple_chain_subckt_names + double_chain_subckt_names:
        net = block.inputs['cin']
        carry_chain_nets[net].append(block_num)
        if 'cout' in block.outputs:
            net = block.outputs['cout']
            carry_chain_nets[net].append(block_num)

In [17]:
carry_chains = []
start_atom_indices = carry_chain_nets['gnd']
carry_chain_nets.pop('gnd')
for start_atom_index in start_atom_indices:
    carry_chain = [start_atom_index]
    atom_index = start_atom_index
    atom_found = True
    while atom_found:
        atom_found = False
        for net, atom_indices in carry_chain_nets.items():
            if atom_index in atom_indices:
                atom_indices.remove(atom_index)
                atom_index = list(atom_indices)[0]
                atom_indices.remove(atom_index)
                carry_chain.append(atom_index)
                selected_net = net
                atom_found = True
        if atom_found:
            carry_chain_nets.pop(selected_net)        
    carry_chains.append(carry_chain)

In [18]:
cc_distribution_simple = []
cc_distribution_double = []
for chain in carry_chains:
    first_block_num = chain[0]
    if 'sharein' in netlist.get_block_graph()[first_block_num].inputs:
        cc_distribution_double.append(len(chain))
    else:
        cc_distribution_simple.append(len(chain))
cc_distribution_double, cc_distribution_simple

([4, 4, 4, 4, 6],
 [6, 21, 63, 5, 18, 46, 34, 74, 76, 33, 18, 33, 34, 34, 17, 17, 34])

check data inputs carry chains

In [19]:
block_graph = netlist.get_block_graph()
net_graph = netlist.get_net_graph()

In [20]:
block_graph[block_num].type

'dffeas'

In [21]:
def dfs_search(block_num, block_graph, net_graph, depth=0):
    block_inp = block_graph[block_num].inputs
    block_type = block_graph[block_num].type
    if block_type != 'dffeas':
        print(depth, block_type)
        for inp in block_inp:
            if 'data' in inp:
                data_net = block_inp[inp]
                if 'source' in net_graph[data_net] and net_graph[data_net]['source'] != []:
                    block_num = list(net_graph[data_net]['source'][0])[0]
                    dfs_search(block_num, block_graph, net_graph, depth+1)


In [22]:
for carry_chain in carry_chains:
    for i, block_num in enumerate(carry_chain):
        print(i)
        dfs_search(block_num, block_graph, net_graph)


0
0 stratixiv_lcell_comb
1 stratixiv_lcell_comb
2 stratixiv_lcell_comb
3 stratixiv_lcell_comb
3 stratixiv_lcell_comb
3 stratixiv_lcell_comb
3 stratixiv_lcell_comb
3 stratixiv_lcell_comb
2 stratixiv_lcell_comb
3 stratixiv_lcell_comb
3 stratixiv_lcell_comb
3 stratixiv_lcell_comb
3 stratixiv_lcell_comb
2 stratixiv_lcell_comb
3 stratixiv_lcell_comb
3 stratixiv_lcell_comb
2 stratixiv_lcell_comb
3 stratixiv_lcell_comb
2 stratixiv_lcell_comb
1
0 stratixiv_lcell_comb
2
0 stratixiv_lcell_comb
3
0 stratixiv_lcell_comb
4
0 stratixiv_lcell_comb
5
0 stratixiv_lcell_comb
0
0 stratixiv_lcell_comb
1
0 stratixiv_lcell_comb
2
0 stratixiv_lcell_comb
3
0 stratixiv_lcell_comb
4
0 stratixiv_lcell_comb
5
0 stratixiv_lcell_comb
6
0 stratixiv_lcell_comb
7
0 stratixiv_lcell_comb
8
0 stratixiv_lcell_comb
9
0 stratixiv_lcell_comb
10
0 stratixiv_lcell_comb
11
0 stratixiv_lcell_comb
12
0 stratixiv_lcell_comb
13
0 stratixiv_lcell_comb
14
0 stratixiv_lcell_comb
15
0 stratixiv_lcell_comb
16
0 stratixiv_lcell_comb
17
0

### Write GNL file

GNL file: library description and circuit description

Maak gates library op basis van de subckt collectie
\
Carry chains verwijderen
\
vcc en gnd poorten verwijderen

In [23]:
rent_exp = 0.60

In [24]:
bm_folder = '../benchmarks/'
file_name_gnl = file_name + f'_gnl_{rent_exp:.2f}'
file_name_hnl = file_name_gnl
path_gnl_file = os.path.join(bm_folder,'gnl' ,file_name_gnl +'.gnl')
path_hnl = os.path.join(bm_folder,'hnl' ,file_name_hnl +'.hnl')
path_blif = os.path.join(bm_folder, 'blif',file_name_hnl +'.blif')

gnl = '../GNL/cmake-build-debug/GNL'

In [25]:
gates_library = {}
for subckt_name, count in subckt_instances.count.items():
    if count:
        subckt = subckt_instances.subckts[subckt_name]
        no_inputs = sum([port_label == PortLabel.GNL for port_label in subckt.in_ports.values()])
        no_outputs = sum([port_label == PortLabel.GNL for port_label in subckt.out_ports.values()])
        latch = subckt.type == 'dffeas'
        gates_library[subckt_name] = {'inputs': no_inputs, 'outputs': no_outputs, 'latch': latch, 'count' : count}

In [26]:
circuit = {
    'latches': 0,
    'gates_distribution': [gate['count'] for gate in gates_library.values()],
    'rent_characteristics': [#{'size': 30, 'p': 0.5, 'q': 0.6}, 
                             {'size': 80000, 'p': rent_exp, 'q': 0.1}, 
                             {'size': 9000, 'I': len(inputs_main), 'O': len(outputs_main)}]
}
circuit['rent_characteristics'][-1]['size'] = circuit['latches'] + sum(circuit['gates_distribution'])

In [27]:
lines = []
lines.append('[library]')
lines.append('name=lib')
# lines.append('latch=dff 1 1')
for gate_name, gate in gates_library.items():
    if gate['latch']:
        lines.append('latch=' + gate_name + ' ' + str(gate['inputs']) + ' ' + str(gate['outputs']))
    else:
        lines.append('gate=' + gate_name + ' ' + str(gate['inputs']) + ' ' + str(gate['outputs']))

lines.append('\n')
lines.append('[circuit]')
lines.append('name=' + file_name_hnl)
lines.append('libraries=lib')
lines.append('distribution=' + ' '.join([str(val) for val in circuit['gates_distribution']]))
for rent in circuit['rent_characteristics']:
    for key, value in rent.items():
        lines.append(key+'='+str(value))

In [28]:
file1 = open(path_gnl_file, 'w')
lines = file1.writelines([entry + '\n' for entry in lines])
file1.close()

### Run GNL

In [29]:
cd '/home/marieke/GNL_clone/benchmarks/hnl'

/home/marieke/GNL_clone/benchmarks/hnl


In [30]:
gnl = '../../GNL/cmake-build-debug/GNL'
path_gnl_file = os.path.join('..','gnl' ,file_name_gnl +'.gnl')
subprocess.run([gnl, path_gnl_file], capture_output=True)

CompletedProcess(args=['../../GNL/cmake-build-debug/GNL', '../gnl/sparcT1_core_stratixiv_arch_timing_gnl_0.60.gnl'], returncode=0, stdout=b'', stderr=b'')

In [31]:
cd '/home/marieke/GNL_clone/Visualizations/'

/home/marieke/GNL_clone/Visualizations


### Convert HNL to blif

In [32]:
file1 = open(path_hnl, 'r')
lines = file1.readlines()
file1.close()

circuit_index = [i for i, line in enumerate(lines) if 'circuit' in line and not '#' in line][0]
circuit_inputs = lines[circuit_index + 1]
circuit_outputs = lines[circuit_index + 2]
block_lines = lines[circuit_index + 3 : -1]

### Build netlist

In [33]:
gen_netlist = Netlist()

Add blocks and gnl generated datapaths

In [34]:
for block_line in block_lines:
    split = block_line.strip().split()
    subckt_name = split[0]
    
    subckt = subckt_instances.subckts[subckt_name]
    type_ = subckt.type
    block = Block(type_, subckt_name, {}, [], {})
    assert block.inputs == {}
    
    data_input_ports = [port for port, label in subckt.in_ports.items() if label == PortLabel.GNL]
    data_output_ports = [port for port, label in subckt.out_ports.items() if label == PortLabel.GNL]
    
    subckt_io = split[1:]
    assert len(subckt_io) == len(data_input_ports) + len(data_output_ports)
    data_input_nets = subckt_io[:len(data_input_ports)]
    data_output_nets = subckt_io[-len(data_output_ports):]
    
    for port, net in zip(data_input_ports, data_input_nets):
        block.add_input(port, net)
        
    for port, net in zip(data_output_ports, data_output_nets):
        block.add_output(port, net)        
    
    for clk_port in subckt.clocks:
        block.add_clk_port(clk_port)
    
    gen_netlist.add_block(block)

Give control nets random sources, decoupled by a dffeas

In [35]:
for sub_type, subckt in subckt_instances.subckts.items():
    if subckt.type == 'dffeas':
        if len(subckt.in_ports) == 1:
            std_dffeas_subckt_name = sub_type
            std_dffeas_clks = subckt.clocks
            std_dffeas_in_port = list(subckt.in_ports)[0]
            std_dffeas_out_port = list(subckt.out_ports)[0]

nets = list(gen_netlist.get_net_graph())
ctrl_net_sources = dict(zip(ctrl_nets, sample(nets, len(ctrl_nets))))

Add blocks constant nets and ctrl nets


LAB and MAC

In [36]:
used_ctrl_nets = set()
for block in gen_netlist.blocks:
    label = subckt_instances.subckts[block.sub_type].label
    if label == SubcktLabel.LAB or label == SubcktLabel.MAC:
        for port, label in subckt_instances.subckts[block.sub_type].in_ports.items():
            if label == PortLabel.CONSTANT:
                net = constant_ports[port]
                block.add_input(port, net)
            if label == PortLabel.CTRL:
                distr = list(ctrl_ports_nets[port].values())
                distr = np.array(distr)/sum(distr)
                net = choice(list(ctrl_ports_nets[port].keys()), 1,
                              p=distr)[0]
                used_ctrl_nets.add(net)
                block.add_input(port, net)

RAM

In [37]:
ram_blocks_per_subckt = defaultdict(list)
for sub_type in ram_ctrl_signatures.keys():
    for block in gen_netlist.get_blocks_with_sub_type(sub_type):
        ram_blocks_per_subckt[sub_type].append(block)
    assert len(ram_blocks_per_subckt[sub_type]) == sum(ram_ctrl_signatures_counts[sub_type])
    
    binned_blocks = np.split(ram_blocks_per_subckt[sub_type], np.cumsum(ram_ctrl_signatures_counts[sub_type])[:-1])
    assert len(binned_blocks) == len(ram_ctrl_signatures[sub_type])
    
    for blocks, signature in zip(binned_blocks, ram_ctrl_signatures[sub_type]):
        for block in blocks:
            for port, net in signature.items():
                block.add_input(port, net)
                used_ctrl_nets.add(net)

Add source flip flops to ctrl nets

In [38]:
for ctrl_net in used_ctrl_nets:
    if ctrl_net in ctrl_net_sources:
        source_net = ctrl_net_sources[ctrl_net]
        ff_block = Block('dffeas', std_dffeas_subckt_name,
                     {std_dffeas_in_port: source_net},
                     std_dffeas_clks,
                     {std_dffeas_out_port:ctrl_net}
                    )
        gen_netlist.add_block(ff_block)

Add carry chain ports

Build combinational netgraph

In [39]:
class NetGraph:

        def __init__(self, vertices):
            # No. of vertices
            self.V = len(vertices)
            self.input_label = len(vertices) * [False]
            self.output_label = len(vertices) * [False]
            
            # default dictionary to store graph
            self.graph = defaultdict(list)

            self.Time = 0
            
            self.vertice_to_alias = dict(zip(list(vertices), np.arange(len(vertices))))
            self.alias_to_vertice = dict(zip(np.arange(len(vertices)), list(vertices)))
            assert self.vertice_to_alias[self.alias_to_vertice[100]] == 100
        
        def set_input(self, u):
            self.input_label[self.vertice_to_alias[u]] = True
            
        def set_output(self, u):
            self.output_label[self.vertice_to_alias[u]] = True
            
        # function to add an edge to graph
        def addEdge(self, u, v):
            self.graph[self.vertice_to_alias[u]].append(self.vertice_to_alias[v])
        
        
        # Prints all not yet visited vertices reachable from s
        def DFS(self, s):           # prints all vertices in DFS manner from a given source.
                                    # Initially mark all vertices as not visited
            visited = [False for i in range(self.V)]

            # Create a stack for DFS
            stack = []

            # Push the current source node.
            stack.append(s)
            
            # Descendants
            descendants = []
            
            while (len(stack)):
                # Pop a vertex from stack and print it
                s = stack[-1]
                stack.pop()

                # Stack may contain same vertex twice. So
                # we need to print the popped item only
                # if it is not visited.
                if (not visited[s]):
                    descendants.append(s)
                    visited[s] = True

                # Get all adjacent vertices of the popped vertex s
                # If a adjacent has not been visited, then push it
                # to the stack.
                for node in self.graph[s]:
                    if (not visited[node]):
                        stack.append(node)
            return descendants
        
        def get_descendants(self, u):
            visited = [False] * (self.V)
            u = self.vertice_to_alias[u]
            descendants = self.DFS(u)
            descendants.remove(u)
            return [self.alias_to_vertice[v] for v in descendants]
            
        def DFSUtil(self, temp, v, visited):
 
            # Mark the current vertex as visited
            visited[v] = True
            # Store the vertex to list
            temp.append(v)

            # Repeat for all vertices adjacent
            # to this vertex v
            for i in self.graph[v]:
                if visited[i] == False:

                    # Update the list
                    temp = self.DFSUtil(temp, i, visited)
                    
            return temp
        
        # Method to retrieve connected components
        # in an undirected graph
        def connectedComponents(self):
            visited = []
            cc = []
            for i in range(self.V):
                visited.append(False)
            for v in range(self.V):
                if visited[v] == False:
                    temp = []
                    cc.append(self.DFSUtil(temp, v, visited))
            connected_components = [[self.alias_to_vertice[v] for v in cc_] for cc_ in cc]
            connected_components_input_labels = [[self.input_label[v] for v in cc_] for cc_ in cc]
            connected_components_output_labels = [[self.output_label[v] for v in cc_] for cc_ in cc]
            return connected_components, connected_components_input_labels, connected_components_output_labels

In [40]:
blocks_comb_subset = []
block_in_comb_subset = defaultdict(bool)
for block in gen_netlist.get_block_graph():
    if block.type != 'dffeas':
        blocks_comb_subset.append(block)
        block_in_comb_subset[block] = True

In [45]:
net_graph = NetGraph(blocks_comb_subset)
net_blocks = netlist.get_net_graph()
blocks = netlist.get_block_graph()
for source_sinks in net_blocks.values():
    block_num = source_sinks['source']
    if block_num != []:
        source_num = list(block_num[0])[0]
        source = blocks[source_num]

        if block_in_comb_subset[source]:
            if source.type != 'stratixiv_lcell_comb':
                print(source.type)
            for block_num in source_sinks['sinks']:
                sink_num = list(block_num)[0]
                sink = blocks[sink_num]

                if block_in_comb_subset[sink]:
                    net_graph.addEdge(source, sink)
                    net_graph.addEdge(sink, source)
                else:
                    net_graph.set_output(source)
                    print('source')
        else: # label sinks inputs
            for block_num in source_sinks['sinks']:
                sink_num = list(block_num)[0]
                sink = blocks[sink_num]
                if block_in_comb_subset[sink]:
                    net_graph.set_input(sink)
                    
                    print('sink')

In [42]:
len(net_graph.connectedComponents())

3

In [43]:
connected_components[0]

NameError: name 'connected_components' is not defined

In [44]:
connected_components = net_graph.connectedComponents()
[len(connected_comp) for connected_comp in connected_components[0]]

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [52]:
net_graph.V

46739

In [12]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

N_points = 100000
n_bins = 20

# Generate two normal distributions
dist1 = rng.standard_normal(N_points)


fig, axs = plt.subplots(1, 2, sharey=True, tight_layout=True)

# We can set the number of bins with the *bins* keyword argument.
axs[0].hist(dist1, bins=n_bins)
axs[1].hist(dist2, bins=n_bins)

NameError: name 'rng' is not defined

In [45]:
sum(net_graph.input_label)

0

In [46]:
connected_components[0]dd

[[<__main__.Block at 0x7f210cc1eb50>,
 [<__main__.Block at 0x7f210cc4e760>, <__main__.Block at 0x7f210cc4e850>],
 [<__main__.Block at 0x7f210cbd9a30>, <__main__.Block at 0x7f210cbd9b50>],
 [<__main__.Block at 0x7f210cbf15e0>, <__main__.Block at 0x7f210cbf16d0>],
 [<__main__.Block at 0x7f210cbfb0d0>, <__main__.Block at 0x7f210cbfb040>],
 [<__main__.Block at 0x7f210cc06a30>, <__main__.Block at 0x7f210cc06a90>],
 [<__main__.Block at 0x7f210cbc0250>, <__main__.Block at 0x7f210cb612e0>],
 [<__main__.Block at 0x7f210cb61bb0>, <__main__.Block at 0x7f210cb61c70>],
 [<__main__.Block at 0x7f210cb6d430>, <__main__.Block at 0x7f210cb6d490>],
 [<__main__.Block at 0x7f210cb78520>, <__main__.Block at 0x7f210cb78610>],
 [<__main__.Block at 0x7f210cb78700>, <__main__.Block at 0x7f210cb787f0>],
 [<__main__.Block at 0x7f210cb8f1c0>, <__main__.Block at 0x7f210cb8f250>],
 [<__main__.Block at 0x7f210cb31790>, <__main__.Block at 0x7f210cb31a00>],
 [<__main__.Block at 0x7f210cae95b0>, <__main__.Block at 0x7f2

collect carry chain blocks and group them

In [95]:
len(np.unique(connected_components[0]))
len(connected_components[1])

3

In [13]:
simple_chain_blocks = []
double_chain_blocks = []
for block in gen_netlist.get_block_graph():
    if block.sub_type in simple_chain_subckt_names:
        simple_chain_blocks.append(block)
    if block.sub_type in double_chain_subckt_names:
        double_chain_blocks.append(block)

NameError: name 'gen_netlist' is not defined

In [14]:
simple_chains = []
double_chains = []

simple_chain_numbers = choice(np.arange(len(cc_distribution_simple)), len(simple_chain_blocks), p=np.array(cc_distribution_simple)/sum(cc_distribution_simple))
double_chain_numbers = choice(np.arange(len(cc_distribution_double)), len(double_chain_blocks), p=np.array(cc_distribution_double)/sum(cc_distribution_double))

simple_chains = [[] for i in range(len(cc_distribution_simple))]
for chain_number, block in zip(simple_chain_numbers, simple_chain_blocks):
    simple_chains[chain_number].append(block)
    

double_chains = [[] for i in range(len(cc_distribution_double))]
for chain_number, block in zip(double_chain_numbers, double_chain_blocks):
    double_chains[chain_number].append(block) 

[len(simple_chain) for simple_chain in simple_chains], [len(double_chain) for double_chain in double_chains]

NameError: name 'cc_distribution_simple' is not defined

In [15]:
len(simple_chains), len(double_chains)

(0, 0)

Order carry chains

In [182]:
def chain_desc(chain):
    chain_desc = defaultdict(list)
    for block in chain:
        descendants = [v for v in net_graph.get_descendants(block)]
        descendants = [descendant for descendant in descendants if descendant in chain]
        chain_desc[block] = descendants
    return chain_desc

In [183]:
# Find block without descendants
# Remove block from chain
# Remove block from other other descendant lists
def order_chain(chain_desc):
    chain_desc = chain_desc.copy()
    unordered = list(chain_desc)
    reverse_ordered = []
    while unordered != []:
        for block in unordered:
            if chain_desc[block] == []:
                break
        reverse_ordered.append(block)
        unordered.remove(block)
        for u in chain_desc.keys():
            chain_desc[u] = [v for v in chain_desc[u] if v != block]
    reverse_ordered.reverse()
    return(reverse_ordered)

In [184]:
chain_blocks_unordered = [block for chain in simple_chains + double_chains for block in chain]
chain_blocks_ordered = order_chain(chain_desc(chain_blocks_unordered))

In [185]:
ordered_simple_chains = []
for chain in simple_chains:
    ordered_simple_chains.append([block for block in chain_blocks_ordered if block in chain])
ordered_double_chains = []
for chain in double_chains:
    ordered_double_chains.append([block for block in chain_blocks_ordered if block in chain])

Wire carry chains

In [186]:
decoupled_input_nets = set()
for i, chain in enumerate(ordered_simple_chains):
    net_carry = 'gnd'
    for j, block in enumerate(chain):
        # decouple inputs and outputs
        for input_port, input_net in block.inputs.items():
            if not input_net in decoupled_input_nets:
                decoupled_input_net = input_net + '_ffi'
                block.add_input(input_port, decoupled_input_net)
                ff_block = Block('dffeas', std_dffeas_subckt_name,
                         {std_dffeas_in_port: input_net},
                         std_dffeas_clks,
                         {std_dffeas_out_port:decoupled_input_net}
                        )
                gen_netlist.add_block(ff_block)
                decoupled_input_nets.add(input_net)

        for output_port, output_net in block.outputs.items():
            decoupled_output_net = output_net + '_ffo'
            block.add_output(output_port, decoupled_output_net)
            ff_block = Block('dffeas', std_dffeas_subckt_name,
                     {std_dffeas_in_port: decoupled_output_net},
                     std_dffeas_clks,
                     {std_dffeas_out_port:output_net}
                    )
            gen_netlist.add_block(ff_block)
            
        block.add_input('cin', net_carry)
        if block != chain[-1]:
            net_carry = f'carry_s{i:03}_{j:03}'
            block.add_output('cout', net_carry)
            

for i, chain in enumerate(ordered_double_chains):
    net_carry = 'gnd'
    net_share = 'gnd'
    for j, block in enumerate(chain):
        # decouple inputs and outputs
        for input_port, input_net in block.inputs.items():
            if not input_net in decoupled_input_nets:
                decoupled_input_net = input_net + '_ffi'
                block.add_input(input_port, decoupled_input_net)
                ff_block = Block('dffeas', std_dffeas_subckt_name,
                         {std_dffeas_in_port: input_net},
                         std_dffeas_clks,
                         {std_dffeas_out_port:decoupled_input_net}
                        )
                decoupled_input_nets.add(input_net)
                gen_netlist.add_block(ff_block)
                
        for output_port, output_net in block.outputs.items():
            decoupled_output_net = output_net + '_ffo'
            block.add_output(output_port, decoupled_output_net)
            ff_block = Block('dffeas', std_dffeas_subckt_name,
                     {std_dffeas_in_port: decoupled_output_net},
                     std_dffeas_clks,
                     {std_dffeas_out_port:output_net}
                    )
            gen_netlist.add_block(ff_block)
        
        block.add_input('cin', net_carry)
        block.add_input('sharein', net_share)
        
        if block != chain[-1]:
            net_carry = f'carry_d{i:03}_{j:03}'
            net_share = f'share_d{i:03}_{j:03}'
            block.add_output('cout', net_carry)
            block.add_output('shareout', net_share)

Add flip flops to carry chain driving inputs

Check depth datainputs carry chains

### Examine and compare netlist

In [187]:
gen_netlist.net_graph()
gen_netgraph = gen_netlist.get_net_graph()
gen_netgraph

defaultdict(<function __main__.Netlist.net_graph.<locals>.<lambda>()>,
            {'n460': defaultdict(list,
                         {'sinks': [{66: 'datac'},
                           {67: 'datac'},
                           {656: 'datae'}]}),
             'n463': defaultdict(list,
                         {'sinks': [{66: 'datab'},
                           {67: 'datab'},
                           {548: 'datab'},
                           {756: 'datac'},
                           {782: 'datae'},
                           {948: 'datac'}]}),
             'n462': defaultdict(list,
                         {'sinks': [{66: 'dataa'}, {67: 'dataa'}]}),
             'n464': defaultdict(list,
                         {'source': [{66: 'combout'}],
                          'sinks': [{67: 'datad'}, {821: 'dataa'}]}),
             'n465': defaultdict(list,
                         {'source': [{67: 'combout'}],
                          'sinks': [{432: 'datab'}]}),
             'n478': de

#### Find all logic clusters

In [188]:
gen_blocks_comb_subset = []
block_in_comb_subset = defaultdict(bool)
for block in gen_netlist.get_block_graph():
    if block.type != 'dffeas':
        gen_blocks_comb_subset.append(block)
        block_in_comb_subset[block] = True

In [189]:
gen_net_graph = NetGraph(gen_blocks_comb_subset)
gen_netlist.net_graph()
net_blocks = gen_netlist.get_net_graph()
blocks = gen_netlist.get_block_graph()
for source_sinks in net_blocks.values():
    block_num = source_sinks['source']
    if block_num != []:
        source_num = list(block_num[0])[0]
        source = blocks[source_num]
        if block_in_comb_subset[source]:
            for block_num in source_sinks['sinks']:
                sink_num = list(block_num)[0]
                sink = blocks[sink_num]
                if block_in_comb_subset[sink]:
                    gen_net_graph.addEdge(source, sink)
                    gen_net_graph.addEdge(sink, source)                    

In [190]:
gen_netlist.net_graph()
gen_netlist.get_block_graph

<bound method Netlist.get_block_graph of <__main__.Netlist object at 0x7f6cfc5637c0>>

In [196]:
gen_netlist.net_graph()

In [191]:
# block_graph = gen_netlist.get_block_graph()
# net_graph = gen_netlist.get_net_graph()

In [192]:
# for carry_chain in ordered_simple_chains:
#     for i, block in enumerate(carry_chain):
#         block_num = block_graph.index(block)
#         print('new search', i, block_num)
#         dfs_search(block_num, block_graph, net_graph)

### Convert netlist to blif

In [166]:
netgraph

In [193]:
lines_out = []

lines_out.append('.model top')
lines_out.append('.inputs ' + ' '.join(circuit_inputs.split()[1:])+ ' gclk')
lines_out.append('.outputs ' + ' '.join(circuit_outputs.split()[1:]))

lines_out.append('\n.names gnd')
lines_out.append('0')
lines_out.append('\n.names vcc')
lines_out.append('1\n')


for block in gen_netlist.get_block_graph():
    line = f'.subckt {block.type}'
    for port, net in block.inputs.items():
        line += f' {port}={net}'
    for clk_port in block.clocks:
        line += f' {clk_port}=gclk'
    for port, net in block.outputs.items():
        line += f' {port}={net}'
        
    lines_out.append(line)
    lines_out.append('')

lines_out.append('.end')

In [194]:
for subckt_model in subckt_models:
    lines_out.append('')
    lines_out.extend([line.strip('\n').strip('\n') for line in subckt_model])

In [87]:
file1 = open(path_blif, 'w')
lines = file1.writelines([entry + '\n' for entry in lines_out])
file1.close()

In [88]:
path_blif

'../benchmarks/blif/sparcT1_core_stratixiv_arch_timing_gnl_0.60.blif'